# Домашно 2

Трениране и подобряване на модели

Предайте същата тетрадка като тази в заданието с нанесените от вас промени. Кръстете файла с факултетния си номер.

# Задача 1

В заданието  имате данните отностно сърдечни заболявания, които са взети от [тук](https://archive.ics.uci.edu/dataset/222/bank+marketing).
Данните, които трябва да прочетете, са файлът `bank.csv`

1.1 Прочетете набора от данни с помощта на `pandas`.

1.2 След което разбийте данните на атрибути и целеви атрибут. в случая целевия атрибут е y.

1.3 Подгответе категорийните променливи, така че да могат да бъдат обработвани като индикатори.

1.4 Разбийте данните на тренировъчно и тестово множество (като 30% от данните да са в тестовото множество). 

Преди да ги разбиете вижте какъв е баланса между двата класа и помислете как трябва да ги разбиете(hint: stratification) 

1.5 Скалирайте данните така, че да имате средно 0 и стандартно отклонение 1.


In [81]:

import numpy as np
import mglearn
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import warnings

from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_moons, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPClassifier

warnings.filterwarnings(action='ignore', module='scipy', message='^internal gelsd')

%matplotlib inline

In [82]:
# 1.1
data = pd.read_csv('bank.csv',sep=';')

In [83]:
# 1.2
X, y = data.loc[:,data.columns!='y'] , data.loc[:,data.columns=='y']
#print("{} {}".format(X.shape,y.shape))


In [84]:
# 1.3
X=pd.get_dummies(X).replace({True: 1, False: 0})
y=y.replace({'no':0 , 'yes':1})
#y['y'].value_counts().loc[1]


In [85]:
# 1.4
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,stratify=y , random_state=619)
# y_train.value_counts() #-> the proportion between the two classes should be around 0.13 (that means we stratify correct with the input data in y)
# y_test.value_counts() #- > the proportion between the two classes should be around 0.13 (that means we stratify correct with the input data in y)


In [86]:
# 1.5
from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

# X_train = pd.DataFrame(X_train,columns=['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
#        'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
#        'job_management', 'job_retired', 'job_self-employed', 'job_services',
#        'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
#        'marital_divorced', 'marital_married', 'marital_single',
#        'education_primary', 'education_secondary', 'education_tertiary',
#        'education_unknown', 'default_no', 'default_yes', 'housing_no',
#        'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
#        'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug',
#        'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
#        'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
#        'poutcome_failure', 'poutcome_other', 'poutcome_success',
#        'poutcome_unknown'])
# X_test = pd.DataFrame(X_test,columns=['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
#        'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
#        'job_management', 'job_retired', 'job_self-employed', 'job_services',
#        'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
#        'marital_divorced', 'marital_married', 'marital_single',
#        'education_primary', 'education_secondary', 'education_tertiary',
#        'education_unknown', 'default_no', 'default_yes', 'housing_no',
#        'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
#        'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug',
#        'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
#        'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
#        'poutcome_failure', 'poutcome_other', 'poutcome_success',
#        'poutcome_unknown'])
# np.mean(X_train.describe().loc['mean',:]) #should be 0
# np.mean(X_test.describe().loc['mean',:]) # should be 0
# np.mean(X_train.describe().loc['std',:]) # should be 1
# np.mean(X_test.describe().loc['std',:]) # should be 1

# Задача 2 Дървета

Използвайте крос-валидация с 5 гънки(5-fold cross validation), за да обучите и оптимизирате хиперпараметрите за дърво на решенията.

Помислете внимателно как трябва да разбиете гънките при толкова небалансирани класове(hint: stratified k-fold cross-validation).

Помислете как да оценявате модела не забравяйте, че точността не работи в този случай, защото данните са силно небалансирани. Имаме нужда от нещо, което съчетава прецизност и обхват(възвръщаемост).

Направете grid search като изберете по 3 стойности за всеки от хипер параметрите
* `max_depth`
* `min_samples_leaf`
* `max_leaf_nodes`

Вижте как се представя алгоритъма върху тренировачните данни и кои са най-добрите хипер параметри.

In [87]:
# 2
from sklearn.model_selection import cross_val_score ,StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


In [88]:
# 2
grid = {
    "max_depth": [6,8,10],
    "min_samples_leaf": [5, 10, 20],
    "max_leaf_nodes": [10,20,30],
}


search = GridSearchCV(DecisionTreeClassifier(random_state=619),grid,cv=5)

search.fit(X_train,y_train)
print(search.best_estimator_)


#cross_val_score izpolzva StratifiedKFold cv po default pri classificaciq (kakto e v nashiq sluchai)
scores = cross_val_score(search,X_train,y_train,cv=5,scoring="f1_weighted") # pri silno nebalnsirani klasove f1_weightted e po-dobrata metrika
print("Scores of Stratified5Fold cv: {}".format(scores))
print("Mean of Stratified5Fold cv: {}".format(scores.mean()))

#moje da e bavnichko za ichislenie poneje pravim mnogo iteracii za vsqka vuzmojna stoinost na grid (around 20sec)


DecisionTreeClassifier(max_depth=8, max_leaf_nodes=20, min_samples_leaf=5,
                       random_state=619)
Scores of Stratified5Fold cv: [0.88408015 0.89929821 0.89246387 0.88908795 0.88822558]
Mean of Stratified5Fold cv: 0.8906311492316654


# Задача 3 Случайна гора

Аналогично на предишната задача но този път със случайна гора

Направете grid search като изберете по 3 стойности за всеки от хипер параметрите
* `n_estimators`
* `max_depth`


In [89]:
# 3
from sklearn.ensemble import RandomForestClassifier

grid = {
    "n_estimators": [5,10,20],
    "max_depth": [4,6,8],
}

search = GridSearchCV(RandomForestClassifier(random_state=619),grid,cv=5)
search.fit(X_train,y_train.values.ravel())
print(search.best_estimator_)


#cross_val_score izpolzva StratifiedKFold cv po default pri classificaciq (kakto e v nashiq sluchai)
scores = cross_val_score(search,X_train,y_train.values.ravel(),cv=5,scoring="f1_weighted") # pri silno nebalnsirani klasove f1_weightted e po-dobrata metrika
print("Scores of Stratified5Fold cv: {}".format(scores))
print("Mean of Stratified5Fold cv: {}".format(scores.mean()))

#moje da e bavnichko za ichislenie poneje pravim mnogo iteracii 
#za vsqka vuzmojna stoinost na grid i se skalira sprqmo stoinosta ( mnogo durveta = mnogo bavno , no po-dobri rezultati) (around 20sec)



RandomForestClassifier(max_depth=8, n_estimators=5, random_state=619)
Scores of Stratified5Fold cv: [0.87472489 0.84726208 0.84825395 0.85574752 0.84504841]
Mean of Stratified5Fold cv: 0.8542073697515606


# Задача 4 SVM

Аналогично на предишната задача но този път със svm

Направете grid search като изберете по 3 стойности за всеки от хипер параметрите
* `C`
* `gamma`


In [90]:
# 4
from sklearn.svm import SVC

grid = {
    "C": [0.1, 1, 10],
    "gamma": ["scale", "auto"],
}

search = GridSearchCV(SVC(random_state=619),grid,cv=5)
search.fit(X_train,y_train.values.ravel())
print(search.best_estimator_)


#cross_val_score izpolzva StratifiedKFold cv po default pri classificaciq (kakto e v nashiq sluchai)
scores = cross_val_score(search,X_train,y_train.values.ravel(),cv=5,scoring="f1_weighted") # pri silno nebalnsirani klasove f1_weightted e po-dobrata metrika
print("Scores of Stratified5Fold cv: {}".format(scores))
print("Mean of Stratified5Fold cv: {}".format(scores.mean()))

#moje da e bavnichko za ichislenie poneje pravim mnogo iteracii 
#za vsqka vuzmojna stoinost na grid i se skalira sprqmo stoinosta (around 1min)


SVC(C=1, random_state=619)
Scores of Stratified5Fold cv: [0.86460584 0.85783236 0.85569523 0.87472489 0.85850967]
Mean of Stratified5Fold cv: 0.8622735974933692


# Задача 5 

Повторете задачи 2-4 като изполвате резултатите получени до сега за да избрете нови стойности на същите хипер параметри за които да правите нов grid search.


In [91]:
# 5.2 

# stari_grid = {
#     "max_depth": [6,8,10],
#     "min_samples_leaf": [5, 10, 20],
#     "max_leaf_nodes": [10,20,30],
# }

grid = {
    "max_depth": [8,10,12],
    "min_samples_leaf": [5,15,20],
    "max_leaf_nodes": [30,40,50],
}


search = GridSearchCV(DecisionTreeClassifier(random_state=619),grid,cv=5)

search.fit(X_train,y_train)
print(search.best_estimator_)


#cross_val_score izpolzva StratifiedKFold cv po default pri classificaciq (kakto e v nashiq sluchai)
scores = cross_val_score(search,X_train,y_train,cv=5,scoring="f1_weighted") # pri silno nebalnsirani klasove f1_weightted e po-dobrata metrika
print("Scores of Stratified5Fold cv: {}".format(scores))
print("Mean of Stratified5Fold cv: {}".format(scores.mean()))

#moje da e bavnichko za ichislenie poneje pravim mnogo iteracii za vsqka vuzmojna stoinost na grid (around 30sec)


DecisionTreeClassifier(max_depth=12, max_leaf_nodes=30, min_samples_leaf=5,
                       random_state=619)
Scores of Stratified5Fold cv: [0.88277926 0.90112718 0.87401765 0.88408015 0.88983769]
Mean of Stratified5Fold cv: 0.886368386552564


Стигнахме до изводът , че предните стойности са оптимални за тези данни, и са по-добри отколкото новите . Като държим повече листа, с по-голяма дълбочина и по-малко примери за тестване преди рязане на дървото достигнахме до изводът че - получаваме по-ниски резултати отколкото преди

In [92]:

# 5.3
from sklearn.ensemble import RandomForestClassifier

# stari_grid = {
#     "n_estimators": [5,10,20],
#     "max_depth": [4,6,8],
# }

grid = {
    "n_estimators": [30,40,50],
    "max_depth": [5,7,9],
}

search = GridSearchCV(RandomForestClassifier(random_state=619),grid,cv=5)
search.fit(X_train,y_train.values.ravel())
print(search.best_estimator_)


#cross_val_score izpolzva StratifiedKFold cv po default pri classificaciq (kakto e v nashiq sluchai)
scores = cross_val_score(search,X_train,y_train.values.ravel(),cv=5,scoring="f1_weighted") # pri silno nebalnsirani klasove f1_weightted e po-dobrata metrika
print("Scores of Stratified5Fold cv: {}".format(scores))
print("Mean of Stratified5Fold cv: {}".format(scores.mean()))

#moje da e bavnichko za ichislenie poneje pravim mnogo iteracii 
#za vsqka vuzmojna stoinost na grid i se skalira sprqmo stoinosta ( mnogo durveta = mnogo bavno , no po-dobri rezultati) (around 20sec)


RandomForestClassifier(max_depth=9, n_estimators=50, random_state=619)
Scores of Stratified5Fold cv: [0.87621106 0.85252849 0.86201376 0.86921685 0.86218385]
Mean of Stratified5Fold cv: 0.8644308015370346


Получаваме по-добри резултати , отколкото преди с новите хиперпараметри, но това звучи логично , колкото повече дървета в гората, толкова по-добре можем да си направим преценка за резултата. Както и по-дълбока дълбочина на дървото ни отвежда до по-голяма склонност за научаване по-добре на тренировъчните данни


In [80]:
# 5.4
from sklearn.svm import SVC

# стари_grid = {
#     "C": [0.1, 1, 10],
#     "gamma": ["scale", "auto"],
# }
grid = {
    "C": [10, 100,0.00001],
    "gamma": ["scale", "auto"],
}

search = GridSearchCV(SVC(random_state=619),grid,cv=5)
search.fit(X_train,y_train.values.ravel())
print(search.best_estimator_)


#cross_val_score izpolzva StratifiedKFold cv po default pri classificaciq (kakto e v nashiq sluchai)
scores = cross_val_score(search,X_train,y_train.values.ravel(),cv=5,scoring="f1_weighted") # pri silno nebalnsirani klasove f1_weightted e po-dobrata metrika
print("Scores of Stratified5Fold cv: {}".format(scores))
print("Mean of Stratified5Fold cv: {}".format(scores.mean()))

#moje da e bavnichko za ichislenie poneje pravim mnogo iteracii 
#za vsqka vuzmojna stoinost na grid i se skalira sprqmo stoinosta (around 50sec)


SVC(C=10, random_state=619)
Scores of Stratified5Fold cv: [0.83054257 0.83054257 0.8813141  0.83054257 0.88270282]
Mean of Stratified5Fold cv: 0.8511289248422859


# Задача 6

Сравнете как се предстявят всички модели и изберете кой е най-добрия


**#6**<br/>
Като заключение след проведените тестове , можем да считаме че **DecisionTreeClassifier** се справя най- добре с подадените данни и спрямо cross-validaciqта и хиперпараметрите max_depth=8, max_leaf_nodes=20, min_samples_leaf=5 получаваме среден резултат от 5те итерации : *0.8906311492316654* 
<br/>
<br/>
<br/>
Като втори най-добър е **RandomForestClassifier** с резултат от 5те итерации на крос валидацията : *0.8644308015370346*
<br/>
<br/>
<br/>
И най - зле се представя **SVC**.